<a href="https://colab.research.google.com/github/jeguns/EP7192/blob/main/Capitulo_5/Unidad_5_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Descripción del caso

Una universidad desea evaluar el servicio de su biblioteca en términos de tiempos de espera, uso del servicio y nivel de satisfacción de los estudiantes. Para ello, se encuestó a 70 estudiantes al finalizar el semestre. Se recolectaron las siguientes variables

- Tiempo\_Espera (en minutos): tiempo que un estudiante espero para ser atendido.

- Libros\_Prestados: número de libros en préstamo por parte del estudiante en el ciclo.

- Satisfecho: respuesta a si están satisfechos con el servicio.

# Lectura de datos

In [ ]:
import pandas as pd
datos = pd.read_csv('datos_u5.csv')
datos.head(10)

# Intervalo de confianza para la media

Existe una confianza del 90% de que el verdadero tiempo medio de espera se encuentre entre los 7.61 y 8.70 minutos.


In [ ]:
tiempos  = datos["Tiempo_Espera"]
n        = len(tiempos)
media    = tiempos.mean()
desv_est = tiempos.std(ddof=1)
sem      = desv_est / n**0.5
from scipy import stats
stats.t.interval(confidence=0.90, df=n-1, loc=media, scale=sem)

In [ ]:
from statsmodels.stats.api import DescrStatsW
DescrStatsW(datos["Tiempo_Espera"]).tconfint_mean(alpha=0.10)

Con un 99% de confianza, se puede afirmar que la cantidad media poblacional de libros prestados en el ciclo se encuentra entre 2.84 y 3.54 libros.


In [ ]:
DescrStatsW(datos["Libros_Prestados"]).tconfint_mean(alpha=0.01)

# Intervalo de confianza para la varianza

Se tiene un 95% de confianza de que la varianza poblacional de los tiempos de espera se encuentre entre los 5.51 y 10.79 minutos$^2$.


In [ ]:
from scipy.stats import chi2
import numpy as np

tiempos  = datos["Tiempo_Espera"]
n        = len(tiempos)
s2       = np.var(tiempos, ddof=1)
conf     = 0.95
alpha    = 1 - conf
chi2_inf = chi2.ppf(alpha / 2, df=n - 1)
chi2_sup = chi2.ppf(1 - alpha / 2, df=n - 1)
ic_var_tiempos = ((n - 1) * s2 / chi2_sup, (n - 1) * s2 / chi2_inf)
ic_var_tiempos

Existe una confianza del 90% de que la verdadera varianza de la cantidad de libros prestados esté contenida en el intervalo de 0.95 a 1.66 libros $^2$


In [ ]:
from scipy.stats import chi2
import numpy as np

libros   = datos["Libros_Prestados"]
n        = len(libros)
s2       = np.var(libros, ddof=1)
conf     = 0.90
alpha    = 1 - conf
chi2_inf = chi2.ppf(alpha / 2, df=n - 1)
chi2_sup = chi2.ppf(1 - alpha / 2, df=n - 1)
ic_var_libros = ((n - 1) * s2 / chi2_sup, (n - 1) * s2 / chi2_inf)
ic_var_libros

# Intervalo de confianza para la desviación estándar

Se tiene un 95% de confianza de que la desviación estándar poblacional de los tiempos de espera se encuentre entre los 2.35 y 3.28 minutos.


In [ ]:
(ic_var_tiempos[0]**0.5, ic_var_tiempos[1]**0.5)

Existe una confianza del 90% de que la verdadera desviación estándar de la cantidad de libros prestados esté contenida en el intervalo de 0.97 a 1.29 libros.


In [ ]:
(ic_var_libros[0]**0.5, ic_var_libros[1]**0.5)

# Intervalo de confianza para la proporción

In [ ]:
n = len(datos)
x = (datos["Satisfecho"] == "Sí").sum()
p = x / n
print("p = ",p)
print("np = ", n*p)
print("n(1-p) = ", n * (1 - p))

Con un nivel de confianza del 96%, se estima que la verdadera proporción de estudiantes usuarios de la biblioteca que están satisfechos se encuentra entre 0.5347 y 0.7617.


In [ ]:
from statsmodels.stats.proportion import proportion_confint
proportion_confint(count=x, nobs=n, alpha=0.04, method='normal')

In [ ]:
proportion_confint(count=x, nobs=n, alpha=0.04, method='wilson')

In [ ]:
proportion_confint(count=x, nobs=n, alpha=0.04, method='beta')

# Intervalo de confianza para la mediana

In [ ]:
# Estimación puntual
datos["Tiempo_Espera"].median()

In [ ]:
pip install bootstrapped -q

In [ ]:
# Estimación intervalar
import bootstrapped.bootstrap as bs
import bootstrapped.stats_functions as bs_stats

np.random.seed(2025)
resultado = bs.bootstrap(values         = datos["Tiempo_Espera"].values,
                         stat_func      = bs_stats.median,
                         num_iterations = 2000,
                         alpha          = 0.05)

resultado.lower_bound, resultado.upper_bound

Existe una confianza del 95% de que la verdadera mediana del tiempo de espera esté contenida en el intervalo de 7.4 a 9.4 minutos


# Intervalo de confianza para el coeficiente de variación

In [ ]:
def coef_variacion(muestra):
    return np.std(muestra, ddof=1) / np.mean(muestra)

# Estimación puntual
cv_hat = coef_variacion(x)

In [ ]:
# Estimación intervalar
R     = 2000
boots = np.empty(R)

np.random.seed(2025)
for i in range(R):
    muestra = np.random.choice(x, size=len(x), replace=True)
    boots[i] = coef_variacion(muestra)

np.quantile(boots, [0.005, 0.995])

Existe una confianza del 99% de que el verdadero coeficiente de variación del número de libros prestados esté contenido en el intervalo de 27.08% a 42.48%
